In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter

# Stockfish Spieler

Dieses Notebook implementiert die Verbindung zur Stockfish-Engine, welche innerhalb eines definierten Zeitlimits nach einem möglichst guten Zug sucht.

In [ ]:
import chess
import chess.engine
from AIBaseClass import ChessAI

class StockfishPlayer(ChessAI):

    def __init__(self, player_name: str, elo=1500) -> None:
        super().__init__(player_name)
        self.engine = chess.engine.SimpleEngine.popen_uci("../lib/stockfish/stockfish.exe")
        self.engine.configure({"UCI_LimitStrength": True, "UCI_Elo": elo})

    def get_next_middle_game_move(self: ChessAI, board: chess.Board) -> chess.Move:
        """Communicate with the stockfish engine over uci."""
        # Give stockfish 100ms to think
        result = self.engine.play(board, chess.engine.Limit(time=.1))
        return result.move